In [7]:
! pip install -q transformers datasets Pillow matplotlib torchvision==0.14.1+cpu torch==1.13.1 "optimum-graphcore>=0.6.0, <0.7.0"  -f https://download.pytorch.org/whl/torch_stable.html
! pip install -q examples-utils[common]@git+https://github.com/graphcore/examples-utils@latest_stable

!apt -qq install git-lfs
!git config --global credential.helper store

Loading extensions from ~/.ipython/extensions is deprecated. We recommend managing extensions like any other Python packages, in site-packages.
In order to improve usability and support for future users, Graphcore would like to collect information about the applications and code being run in this notebook. The following information will be anonymised before being sent to Graphcore: 
	- User progression through the notebook 
	- Notebook details: number of cells, code being run and the output of the cells 
	- Environment details 

You can disable logging at any time by running `%unload_ext gc_logger` from any cell. 

git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


In [8]:
import os
os.chdir("/notebooks/Imagenette/Imagenette2Challenge")

In [9]:
import importlib
from transformers import AutoConfig, ResNetForImageClassification, AutoModelForImageClassification
import torch
import torch.nn as nn
import torchvision
from datasets import load_dataset
import torchvision.transforms as transforms
import poptorch
from tqdm import tqdm

import src.utils.helpermethods as helpermethods

In [10]:
executable_cache_dir = os.getenv("POPLAR_EXECUTABLE_CACHE_DIR", "/tmp/exe_cache/") + "/image_classification"
dataset_dir = os.getenv("DATASETS_DIR", "./")

In [11]:
# model_name = "microsoft/resnet-50"
# pretrained_model = ResNetForImageClassification.from_pretrained(model_name)
# pretrained_model.classifier[1] = nn.Linear(in_features=2048, out_features=10)

model_name = "google/vit-base-patch16-224-in21k"
pretrained_model = AutoModelForImageClassification.from_pretrained(
    model_name, num_labels=10)


Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
num_params = sum(p.numel() for p in pretrained_model.parameters() if p.requires_grad)
num_params = format(num_params, ",d").replace(",", ".")
print(num_params)

85.806.346


In [13]:
from optimum.graphcore import IPUTrainingArguments, IPUTrainer, IPUConfig

In [14]:
micro_batch_size = 1 
gradient_accumulation_steps = 32

In [15]:

args = IPUTrainingArguments(
    output_dir="./Checkpoints/"
    f"{model_name}-finetuned-imagenette",
    remove_unused_columns=False,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=micro_batch_size,
    per_device_eval_batch_size=micro_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    num_train_epochs=10,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    n_ipu=4,
    dataloader_drop_last=True,
    push_to_hub=False,
    # model_hub_id = f"username-or-organization/{model_name}-finetuned-eurosat"
)

In [16]:
# import optimum.graphcore as optimum_graphcore
# ipu_config = optimum_graphcore.IPUConfig.from_pretrained(
#     "Graphcore/vit-base-ipu", executable_cache_dir=executable_cache_dir
# )
# print(ipu_config) 

In [17]:
resnet_layers = [3,4,6,3]
vit_layers = [3,3,3,3]
ipu_config = IPUConfig(
    auto_loss_scaling=True,
    executable_cache_dir=executable_cache_dir,
    replication_factor=1,
    inference_replication_factor=1,
    gradient_accumulation_steps=32,
    layers_per_ipu=vit_layers,
    inference_layers_per_ipu=vit_layers,
    ipus_per_replica=4,
    inference_ipus_per_replica=4,
    optimizer_state_offchip=True,
    replicated_tensor_sharding=False,
    matmul_proportion=0.2,
    inference_matmul_proportion=0.2,
    enable_half_partials=True,
)


In [18]:
base_path = "./"
CustomDataset_module = importlib.import_module("src.dataLoaders.CustomDatasetRaw")
file_training = base_path + "Datasets/imagenette2/train.txt"
file_validation = base_path + "Datasets/imagenette2/val.txt"
transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)


In [19]:
train_ds = helpermethods.load_dataset(
        CustomDataset_module, file_training, transform,
    )
val_ds = helpermethods.load_dataset(
        CustomDataset_module, file_validation, transform,
    )

In [20]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    logits = torch.tensor(logits).float()  # Convert to tensor
    labels = torch.tensor(labels).float()  # Convert labels to tensor
    _, predicted = torch.topk(logits, k=1, dim=1)  # adjust k as needed
    correct = torch.eq(predicted, labels.view(-1, 1).expand_as(predicted)).sum().item()
    accuracy = float(correct) / labels.size(0)
    return {"accuracy": accuracy}


In [21]:
def custom_data_collator(batch):
    images = torch.stack([item[0] for item in batch])
    labels = torch.stack([item[1] for item in batch])
    return {"pixel_values": images, "labels": labels}


In [22]:
trainer = IPUTrainer(
    pretrained_model,
    ipu_config,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    compute_metrics=compute_metrics,
    data_collator= custom_data_collator,
)

Overriding IPU config: gradient_accumulation_steps=32
---------- Device Allocation -----------
Embedding  --> IPU 0
Encoder 0  --> IPU 0
Encoder 1  --> IPU 0
Encoder 2  --> IPU 0
Encoder 3  --> IPU 1
Encoder 4  --> IPU 1
Encoder 5  --> IPU 1
Encoder 6  --> IPU 2
Encoder 7  --> IPU 2
Encoder 8  --> IPU 2
Encoder 9  --> IPU 3
Encoder 10 --> IPU 3
Encoder 11 --> IPU 3
Head       --> IPU 3
---------------------------------------


In [23]:
train_results = trainer.train()
# rest is optional but nice to have
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

---------- Device Allocation -----------
Embedding  --> IPU 0
Encoder 0  --> IPU 0
Encoder 1  --> IPU 0
Encoder 2  --> IPU 0
Encoder 3  --> IPU 1
Encoder 4  --> IPU 1
Encoder 5  --> IPU 1
Encoder 6  --> IPU 2
Encoder 7  --> IPU 2
Encoder 8  --> IPU 2
Encoder 9  --> IPU 3
Encoder 10 --> IPU 3
Encoder 11 --> IPU 3
Head       --> IPU 3
---------------------------------------
Compiling Model...
Graph compilation: 100%|██████████| 100/100 [00:08<00:00]
Compiled/Loaded model in 22.184907204005867 secs
***** Running training *****
  Num examples = 9296
  Num epochs = 10
  Instantaneous batch size per device = 1
  Total training batch size (w. parallel, distributed & accumulation) = 32
  Gradient accumulation steps = 32
  Total optimization steps = 2900


  0%|          | 0/2900 [00:00<?, ?it/s]

{'loss': 2.2662, 'learning_rate': 1.724137931034483e-06, 'epoch': 0.03}
{'loss': 2.2666, 'learning_rate': 3.448275862068966e-06, 'epoch': 0.07}
{'loss': 2.2687, 'learning_rate': 5.172413793103448e-06, 'epoch': 0.1}
{'loss': 2.2443, 'learning_rate': 6.896551724137932e-06, 'epoch': 0.14}
{'loss': 2.2086, 'learning_rate': 8.620689655172414e-06, 'epoch': 0.17}
{'loss': 2.1273, 'learning_rate': 1.0344827586206897e-05, 'epoch': 0.21}
{'loss': 2.1348, 'learning_rate': 1.206896551724138e-05, 'epoch': 0.24}
{'loss': 2.101, 'learning_rate': 1.3793103448275863e-05, 'epoch': 0.28}
{'loss': 2.0103, 'learning_rate': 1.5517241379310346e-05, 'epoch': 0.31}
{'loss': 1.8209, 'learning_rate': 1.7241379310344828e-05, 'epoch': 0.34}
{'loss': 1.7312, 'learning_rate': 1.896551724137931e-05, 'epoch': 0.38}
{'loss': 1.4977, 'learning_rate': 2.0689655172413793e-05, 'epoch': 0.41}
{'loss': 1.3434, 'learning_rate': 2.2413793103448276e-05, 'epoch': 0.45}
{'loss': 1.2613, 'learning_rate': 2.413793103448276e-05, 'ep

---------- Device Allocation -----------
Embedding  --> IPU 0
Encoder 0  --> IPU 0
Encoder 1  --> IPU 0
Encoder 2  --> IPU 0
Encoder 3  --> IPU 1
Encoder 4  --> IPU 1
Encoder 5  --> IPU 1
Encoder 6  --> IPU 2
Encoder 7  --> IPU 2
Encoder 8  --> IPU 2
Encoder 9  --> IPU 3
Encoder 10 --> IPU 3
Encoder 11 --> IPU 3
Head       --> IPU 3
---------------------------------------
Compiling Model...
/usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py:1432: UserWarning: Positional args are being deprecated, use kwargs instead. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(

Graph compilation: 100%|██████████| 100/100 [00:02<00:00]
Compiled/Loaded model in 11.808537313714623 secs
***** Running Evaluation *****
  Num examples = 3856
  Batch size = 1


  0%|          | 0/3856 [00:00<?, ?it/s]

...

{'eval_loss': nan, 'eval_accuracy': 0.10217842323651452, 'eval_runtime': 37.4203, 'eval_samples_per_second': 103.046, 'eval_steps_per_second': 103.046, 'epoch': 10.0}


Saving model checkpoint to ./Checkpoints/google/vit-base-patch16-224-in21k-finetuned-imagenette/checkpoint-2900
/usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py:1432: UserWarning: Positional args are being deprecated, use kwargs instead. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(
---------- Device Allocation -----------
Embedding  --> IPU 0
Encoder 0  --> IPU 0
Encoder 1  --> IPU 0
Encoder 2  --> IPU 0
Encoder 3  --> IPU 1
Encoder 4  --> IPU 1
Encoder 5  --> IPU 1
Encoder 6  --> IPU 2
Encoder 7  --> IPU 2
Encoder 8  --> IPU 2
Encoder 9  --> IPU 3
Encoder 10 --> IPU 3
Encoder 11 --> IPU 3
Head       --> IPU 3
---------------------------------------
Configuration saved in ./Checkpoints/google/vit-base-patch16-224-in21k-finetuned-imagenette/checkpoint-2900/ipu_config.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./Check

{'train_runtime': 2163.0657, 'train_samples_per_second': 42.902, 'train_steps_per_second': 1.341, 'train_loss': nan, 'epoch': 10.0}


Saving model checkpoint to ./Checkpoints/google/vit-base-patch16-224-in21k-finetuned-imagenette
/usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py:1432: UserWarning: Positional args are being deprecated, use kwargs instead. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(
---------- Device Allocation -----------
Embedding  --> IPU 0
Encoder 0  --> IPU 0
Encoder 1  --> IPU 0
Encoder 2  --> IPU 0
Encoder 3  --> IPU 1
Encoder 4  --> IPU 1
Encoder 5  --> IPU 1
Encoder 6  --> IPU 2
Encoder 7  --> IPU 2
Encoder 8  --> IPU 2
Encoder 9  --> IPU 3
Encoder 10 --> IPU 3
Encoder 11 --> IPU 3
Head       --> IPU 3
---------------------------------------
Configuration saved in ./Checkpoints/google/vit-base-patch16-224-in21k-finetuned-imagenette/ipu_config.json


***** train metrics *****
  epoch                    =       10.0
  train_loss               =        nan
  train_runtime            = 0:36:03.06
  train_samples_per_second =     42.902
  train_steps_per_second   =      1.341


In [28]:
metrics = trainer.evaluate()
# some nice to haves:
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

***** Running Evaluation *****
  Num examples = 3856
  Batch size = 1


  0%|          | 0/3856 [00:00<?, ?it/s]

***** eval metrics *****
  epoch                   =        1.0
  eval_accuracy           =     0.9839
  eval_loss               =     0.3167
  eval_runtime            = 0:00:52.71
  eval_samples_per_second =     73.141
  eval_steps_per_second   =     73.141


## Inference



In [24]:
import os
os.chdir("/notebooks/Imagenette/Imagenette2Challenge")
import importlib
import torch
from tqdm import tqdm as tqdm
import src.utils.helpermethods as helpermethods
from torchvision import transforms
from transformers import AutoConfig, AutoModelForImageClassification

# Load the model configuration
config = AutoConfig.from_pretrained("./Checkpoints/google/vit-base-patch16-224-in21k-finetuned-imagenette/config.json")
# Load the model
model = AutoModelForImageClassification.from_pretrained("./Checkpoints/google/vit-base-patch16-224-in21k-finetuned-imagenette/pytorch_model.bin", config=config)
model.eval()
labels = ['tench', 'English springer', 'cassette player', 'chain saw', 'church', 'French horn', 'garbage truck', 'gas pump', 'golf ball', 'parachute']

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

base_path = "./"
CustomDataset_module = importlib.import_module("src.dataLoaders.CustomDatasetRaw")
file_training = base_path + "Datasets/imagenette2/train.txt"
file_validation = base_path + "Datasets/imagenette2/val.txt"

val_dataset = helpermethods.load_dataset(
    CustomDataset_module,
    file_validation,
    transform,
)
val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=32, shuffle=True, num_workers=6
)

correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for inputs,labels in tqdm(val_loader):
        outputs = model(inputs).logits
        _, predicted = torch.topk(outputs, k=1, dim=1)  # adjust k as needed
        correct = torch.eq(predicted, labels.view(-1, 1).expand_as(predicted)).sum().item()
        correct_predictions += correct
        total_predictions += labels.size(0)
accuracy = correct_predictions / total_predictions
print(f'Accuracy : {accuracy:.2f}')


100%|██████████| 121/121 [06:44<00:00,  3.34s/it]


Accuracy : 0.99


In [6]:
import random
labels = ['tench', 'English springer', 'cassette player', 'chain saw', 'church', 'French horn', 'garbage truck', 'gas pump', 'golf ball', 'parachute']
num_samples = 5
indices = random.sample(range(len(val_dataset)), num_samples)
samples = [val_dataset[i] for i in indices]

with torch.no_grad():
    for image, label in samples:
        pred_idx = torch.argmax(model(image.unsqueeze(0)).logits).item()
        label = labels[label.item()]
        pred = labels[pred_idx]
        print(f'Label : {label}, pred : {pred}')

Label : parachute, pred : parachute
Label : golf ball, pred : golf ball
Label : gas pump, pred : gas pump
Label : tench, pred : tench
Label : church, pred : church


In [6]:
import os
os.chdir("/notebooks/Imagenette/Imagenette2Challenge")
import importlib
import torch
from tqdm import tqdm as tqdm
import src.utils.helpermethods as helpermethods
from torchvision import transforms
from transformers import AutoConfig, AutoModelForImageClassification
import poptorch


def compute_accuracy_IPU(output: torch.Tensor, labels: torch.Tensor, device_iterations: int) -> float:
    # Reshape labels to match output's batch size
    labels = labels.view(device_iterations, -1, 1)
    _, predicted = torch.topk(output, k=1, dim=1)
    # Calculate the number of correct predictions for each mini-batch and sum them up
    correct = torch.eq(predicted, labels).sum().item()
    return float(correct)

opts = poptorch.Options()
opts.deviceIterations(1)

# Load the model configuration
config = AutoConfig.from_pretrained("./Checkpoints/google/vit-base-patch16-224-in21k-finetuned-imagenette/config.json")
# Load the model
model = AutoModelForImageClassification.from_pretrained("./Checkpoints/google/vit-base-patch16-224-in21k-finetuned-imagenette/pytorch_model.bin", config=config)
model.eval()

labels = ['tench', 'English springer', 'cassette player', 'chain saw', 'church', 'French horn', 'garbage truck', 'gas pump', 'golf ball', 'parachute']

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

base_path = "./"
CustomDataset_module = importlib.import_module("src.dataLoaders.CustomDatasetRaw")
file_training = base_path + "Datasets/imagenette2/train.txt"
file_validation = base_path + "Datasets/imagenette2/val.txt"

val_dataset = helpermethods.load_dataset(
    CustomDataset_module,
    file_validation,
    transform,
)
val_loader = poptorch.DataLoader(options=opts, dataset=val_dataset, batch_size=32, shuffle=True, drop_last=True)

poptorch_inference_model = poptorch.inferenceModel(model, options=val_loader.options)


with torch.no_grad():
    for inputs, labels in tqdm(val_loader):
        outputs = poptorch_inference_model(inputs)
        _, predicted = torch.topk(outputs, k=1, dim=1)  # adjust k as needed
        correct = torch.eq(predicted, labels.view(-1, 1).expand_as(predicted)).sum().item()
        correct_predictions += correct
        total_predictions += labels.size(0)

accuracy = correct_predictions / total_predictions
print(f'Accuracy : {accuracy:.2f}')

  0%|          | 0/120 [06:18<?, ?it/s]


Error: In unknown:0: 'popart_exception': /opt/jenkins/workspace/poplar/poplar_ci_ubuntu_20_04_unprivileged/popart/willow/src/popx/irlowering.cpp:3523 Out of memory on tile 0: 773304 bytes used but tiles only have 638976 bytes of memory
Error raised in:
  [0] popart::Session::prepareDevice: Poplar compilation
  [1] Compiler::compileAndPrepareDevice
  [2] LowerToPopart::compile
  [3] compileWithManualTracing
